In [2]:
import pandas as pd
from igraph import Graph
import disease_process_proteins
import metrics_functions
import importlib
import numpy as np
from ast import literal_eval
from tqdm.notebook import tqdm

In [3]:
reactome_proteins_indexes_apid_huri_df = pd.read_csv("../../data/processed/reactome_proteins_indexes_apid_huri.csv", sep=',', header=0)
disgenet_proteins_indexes_apid_huri_df = pd.read_csv("../../data/processed/disgenet_prot_index_main_comp.csv", sep=',', header=0)
disgenet_prot_index_main_comp = pd.read_csv("../../data/processed/disgenet_proteins_indexes_apid_huri_filtered.csv", sep=',', header=0)
protein_ids_apid_huri_df = pd.read_csv("../../data/processed/protein_ids_apid_huri.csv", sep=',', header=0)
adj_matrix_apid_huri = np.load("../../data/processed/adjacency_matrix_apid_huri.npy")
ppi_80 = np.load("../../data/processed/ppis/ppis_red80_apid_huri.npy", allow_pickle=True)
ppi_80_protein = np.load("../../data/processed/ppis/ppis_red_protein80_apid_huri.npy", allow_pickle=True)

In [4]:
reactome_proteins_indexes_apid_huri_df['protein_index'] = reactome_proteins_indexes_apid_huri_df['protein_index'].apply(literal_eval)
disgenet_proteins_indexes_apid_huri_df['protein_index'] = disgenet_proteins_indexes_apid_huri_df['protein_index'].apply(literal_eval)
reactome_proteins_indexes_apid_huri_df['proteins_ids'] = reactome_proteins_indexes_apid_huri_df['proteins_ids'].apply(literal_eval)
disgenet_proteins_indexes_apid_huri_df['proteins_ids'] = disgenet_proteins_indexes_apid_huri_df['proteins_ids'].apply(literal_eval)
disgenet_prot_index_main_comp['protein_index'] = disgenet_prot_index_main_comp['protein_index'].apply(literal_eval)
disgenet_proteins_indexes_apid_huri_df['conservative_module'] = disgenet_proteins_indexes_apid_huri_df['conservative_module'].apply(literal_eval)
disgenet_proteins_indexes_apid_huri_df['added_nodes'] = disgenet_proteins_indexes_apid_huri_df['added_nodes'].apply(literal_eval)
disgenet_prot_index_main_comp['proteins_ids'] = disgenet_prot_index_main_comp['proteins_ids'].apply(literal_eval)
disgenet_proteins_indexes_apid_huri_df['conservative_module_ids'] = disgenet_proteins_indexes_apid_huri_df['conservative_module_ids'].apply(literal_eval)
disgenet_proteins_indexes_apid_huri_df['added_nodes_ids'] = disgenet_proteins_indexes_apid_huri_df['added_nodes_ids'].apply(literal_eval)

In [5]:
graph_apid_huri = Graph.Read_GML("../../data/processed/graph_apid_huri")

In [6]:
disgenet_prot_index_conservative_module = disgenet_proteins_indexes_apid_huri_df[['process', 'conservative_module_ids', 'conservative_module', 'len_cm']]
disgenet_prot_index_conservative_module.columns = ['process', 'proteins_ids', 'protein_index', 'n_proteins']

# 1. Target Process Metrics

## 1.1. Hypergeometric Test

For a given protein and given process, the number of interactions formed between the protein and the proteins of the process are summed. The hypergeometric test is calculated given this number, the number total number of interactions of the protein,the number of proteins of the process and the total number of proteins. For an easier analysis, the p-values are presented in a -log10 form (higher -log10 means lower p-value).

In [ ]:
importlib.reload(metrics_functions)
process_hypergeometric = metrics_functions.hypergeometric_test(graph_apid_huri, reactome_proteins_indexes_apid_huri_df, adj_matrix_apid_huri)
process_hypergeometric_df = pd.DataFrame.from_dict(process_hypergeometric, orient='index')
process_hypergeometric_df.to_csv('../../data/processed/metrics/process_hypergeometric_apid_huri.csv')
process_hypergeometric_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_hypergeometric = metrics_functions.hypergeometric_test(graph_apid_huri, disgenet_prot_index_main_comp, adj_matrix_apid_huri)
disease_hypergeometric_df = pd.DataFrame.from_dict(disease_hypergeometric, orient='index')
disease_hypergeometric_df.to_csv('../../data/processed/metrics/disease_hypergeometric_apid_huri.csv')
disease_hypergeometric_df.head()

In [54]:
importlib.reload(metrics_functions)
disease_hypergeometric_conservative = metrics_functions.hypergeometric_test(graph_apid_huri, disgenet_prot_index_conservative_module, adj_matrix_apid_huri)
disease_hypergeometric_conservative_df = pd.DataFrame.from_dict(disease_hypergeometric_conservative, orient='index')
disease_hypergeometric_conservative_df.to_csv('../../data/processed/metrics/disease_hypergeometric_conservative_apid_huri.csv')
disease_hypergeometric_conservative_df.head()

  0%|          | 0/17204 [00:00<?, ?it/s]

,C0000786,C0000822,C0001418,C0001787,C0001973,C0002152,C0002395,C0002736,C0003873,C0004096,...,C4317123,C4505436,C4505456,C4552091,C4552766,C4704862,C4707243,C4721453,C4721507,C4722327
A1BG,0.113049,0.113049,0.121945,0.066418,0.272681,0.116385,0.096382,0.063092,0.184327,0.081949,...,0.113049,0.109715,5.260844,0.135295,0.113049,0.135295,0.055332,0.057549,0.076401,0.056441
A1CF,2.017091,2.017091,0.166906,0.090907,0.373220,0.159296,0.131918,0.086354,0.252289,0.112164,...,0.154731,0.150167,0.151688,0.185178,2.017091,0.185178,0.075733,0.078767,0.104570,0.077250
A2M,0.997753,0.997753,2.871837,0.529891,1.301610,0.995197,3.383017,0.503352,1.386556,1.072187,...,3.030981,3.095373,1.825588,1.079405,0.997753,1.079405,0.441446,1.231635,2.301265,2.753903
A2ML1,0.071385,0.071385,0.077002,0.041940,0.172185,0.073491,0.060861,0.039839,0.116393,0.051747,...,0.071385,0.069279,0.069981,0.085432,0.071385,0.085432,0.034940,0.036339,0.048243,0.035639
A4GALT,0.023789,0.023789,0.025661,0.013977,0.057381,0.024491,0.020282,0.013277,0.038789,0.017245,...,0.023789,0.023088,0.023322,0.028471,0.023789,0.028471,0.011644,0.012110,0.016077,0.011877


## 1.2 Shortest Paths

In [ ]:
importlib.reload(metrics_functions)
process_shortest_paths = metrics_functions.shortest_paths(graph_apid_huri, reactome_proteins_indexes_apid_huri_df)
process_shortest_paths_df = pd.DataFrame.from_dict(process_shortest_paths)
process_shortest_paths_df = process_shortest_paths_df.rename(index=dict(zip(list(process_shortest_paths_df.index),list(graph_apid_huri.vs['name']))))
process_shortest_paths_df.to_csv('../../data/processed/metrics/process_shortest_paths_apid_huri.csv')
print(process_shortest_paths_df.shape)
process_shortest_paths_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_shortest_paths = metrics_functions.shortest_paths(graph_apid_huri, disgenet_prot_index_main_comp)
disease_shortest_paths_df = pd.DataFrame.from_dict(disease_shortest_paths)
disease_shortest_paths_df = disease_shortest_paths_df.rename(index=dict(zip(list(disease_shortest_paths_df.index),list(graph_apid_huri.vs['name']))))
disease_shortest_paths_df.to_csv('../../data/processed/metrics/disease_shortest_paths_apid_huri.csv')
print(disease_shortest_paths_df.shape)
disease_shortest_paths_df.head()

In [55]:
importlib.reload(metrics_functions)
disease_shortest_paths_conservative = metrics_functions.shortest_paths(graph_apid_huri, disgenet_prot_index_conservative_module)
disease_shortest_paths_conservative_df = pd.DataFrame.from_dict(disease_shortest_paths_conservative)
disease_shortest_paths_conservative_df = disease_shortest_paths_conservative_df.rename(index=dict(zip(list(disease_shortest_paths_conservative_df.index),list(graph_apid_huri.vs['name']))))
disease_shortest_paths_conservative_df.to_csv('../../data/processed/metrics/disease_shortest_paths_conservative_apid_huri.csv')
print(disease_shortest_paths_conservative_df.shape)
disease_shortest_paths_conservative_df.head()

  0%|          | 0/5321 [00:00<?, ?it/s]

(17204, 5321)


,DCLK3,SUGP1,HNF1B,TKFC,TRERF1,SETD7,CDK3,TPPP,SDK2,MRPS9,...,CCR10,CAV2,SLC2A2,SMAP,CNKSR1,TGFBI,PPP2R2A,CTSB,ADCYAP1R1,RBBP9
A1BG,3,3,3,3,3,1,2,2,3,3,...,3,3,3,3,3,3,3,2,3,3
A1CF,4,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,2,3,4
A2M,2,3,3,3,2,2,2,3,3,2,...,2,3,3,1,3,3,2,2,2,3
A2ML1,4,3,3,3,4,3,3,3,3,3,...,3,3,3,3,4,3,3,3,4,4
A4GALT,4,4,4,3,4,3,3,3,3,3,...,4,3,3,4,4,4,3,3,3,4


# 1.3 Closeness

In [ ]:
importlib.reload(metrics_functions)
process_closeness = metrics_functions.closeness(process_shortest_paths_df, reactome_proteins_indexes_apid_huri_df)
process_closeness_df = pd.DataFrame.from_dict(process_closeness)
process_closeness_df = process_closeness_df.rename(index=dict(zip(list(process_closeness_df.index),list(graph_apid_huri.vs['name']))))
process_closeness_df.to_csv('../../data/processed/metrics/process_closeness_apid_huri.csv')
print(process_closeness_df.shape)
process_closeness_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_closeness = metrics_functions.closeness(disease_shortest_paths_df, disgenet_prot_index_main_comp)
disease_closeness_df = pd.DataFrame.from_dict(disease_closeness)
disease_closeness_df = disease_closeness_df.rename(index=dict(zip(list(disease_closeness_df.index),list(graph_apid_huri.vs['name']))))
disease_closeness_df.to_csv('../../data/processed/metrics/disease_closeness_apid_huri.csv')
print(disease_closeness_df.shape)
disease_closeness_df.head()

In [57]:
importlib.reload(metrics_functions)
disease_closeness_conservative = metrics_functions.closeness(disease_shortest_paths_conservative_df, disgenet_prot_index_conservative_module)
disease_closeness_conservative_df = pd.DataFrame.from_dict(disease_closeness_conservative)
disease_closeness_conservative_df = disease_closeness_conservative_df.rename(index=dict(zip(list(disease_closeness_conservative_df.index),list(graph_apid_huri.vs['name']))))
disease_closeness_conservative_df.to_csv('../../data/processed/metrics/disease_closeness_conservative_apid_huri.csv')
print(disease_closeness_conservative_df.shape)
disease_closeness_conservative_df.head()

(17204, 301)


,C0000786,C0000822,C0001418,C0001787,C0001973,C0002152,C0002395,C0002736,C0003873,C0004096,...,C4317123,C4505436,C4505456,C4552091,C4552766,C4704862,C4707243,C4721453,C4721507,C4722327
A1BG,0.309091,0.309091,0.335366,0.333333,0.312500,0.325077,0.335907,0.325714,0.316190,0.321739,...,0.313846,0.313291,0.347222,0.327078,0.309091,0.327078,0.326797,0.329114,0.323944,0.331169
A1CF,0.327974,0.327974,0.339506,0.338983,0.325365,0.327103,0.341176,0.337278,0.336714,0.324561,...,0.318750,0.320388,0.343643,0.337017,0.327974,0.337017,0.340136,0.331210,0.320930,0.340000
A2M,0.383459,0.383459,0.404412,0.410959,0.374046,0.386029,0.406542,0.380000,0.379863,0.377551,...,0.383459,0.385214,0.389105,0.374233,0.383459,0.374233,0.409836,0.400000,0.379121,0.408000
A2ML1,0.291429,0.291429,0.315186,0.312500,0.298780,0.300860,0.308511,0.304813,0.296429,0.299595,...,0.299120,0.300000,0.313480,0.308081,0.291429,0.308081,0.312500,0.309524,0.302632,0.314815
A4GALT,0.274194,0.274194,0.296496,0.291262,0.282910,0.290859,0.294915,0.295337,0.279933,0.285714,...,0.287324,0.286127,0.299401,0.288416,0.274194,0.288416,0.289017,0.298851,0.293617,0.289773


## 1.4 Betweenness

In [ ]:
importlib.reload(metrics_functions)
process_betweenness = metrics_functions.betweenness(process_shortest_paths_df, reactome_proteins_indexes_apid_huri_df, graph_apid_huri)
process_betweenness_df = pd.DataFrame.from_dict(process_betweenness)
process_betweenness_df = process_betweenness_df.rename(index=dict(zip(list(process_betweenness_df.index),list(graph_apid_huri.vs['name']))))
process_betweenness_df.fillna(value=0, inplace=True)
process_betweenness_df.to_csv('../../data/processed/metrics/process_betweenness_apid_huri.csv')
print(process_betweenness_df.shape)
process_betweenness_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_betweenness = metrics_functions.betweenness(disease_shortest_paths_df, disgenet_prot_index_main_comp, graph_apid_huri)
disease_betweenness_df = pd.DataFrame.from_dict(disease_betweenness)
disease_betweenness_df = disease_betweenness_df.rename(index=dict(zip(list(disease_betweenness_df.index),list(graph_apid_huri.vs['name']))))
disease_betweenness_df.fillna(value=0, inplace=True)
disease_betweenness_df.to_csv('../../data/processed/metrics/disease_betweenness_apid_huri.csv')
print(disease_betweenness_df.shape)
disease_betweenness_df.head()

In [58]:
importlib.reload(metrics_functions)
disease_betweenness_conservative = metrics_functions.betweenness(disease_shortest_paths_conservative_df, disgenet_prot_index_conservative_module, graph_apid_huri)
disease_betweenness_conservative_df = pd.DataFrame.from_dict(disease_betweenness_conservative)
disease_betweenness_conservative_df = disease_betweenness_conservative_df.rename(index=dict(zip(list(disease_betweenness_conservative_df.index),list(graph_apid_huri.vs['name']))))
disease_betweenness_conservative_df.fillna(value=0, inplace=True)
disease_betweenness_conservative_df.to_csv('../../data/processed/metrics/disease_betweenness_conservative_apid_huri.csv')
print(disease_betweenness_conservative_df.shape)
disease_betweenness_conservative_df.head()

  0%|          | 0/301 [00:00<?, ?it/s]

(17204, 301)


,C0000786,C0000822,C0001418,C0001787,C0001973,C0002152,C0002395,C0002736,C0003873,C0004096,...,C4317123,C4505436,C4505456,C4552091,C4552766,C4704862,C4707243,C4721453,C4721507,C4722327
A1BG,0.019802,0.019802,0.000500,0.000000,0.000000,0.000366,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.019802,0.000000,0.000000,0.000000,0.000000,0.000000
A1CF,0.020384,0.020384,0.000667,0.000000,0.000033,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.020384,0.000000,0.041633,0.005279,0.000000,0.007059
A2M,0.045622,0.045622,0.018182,0.005650,0.001204,0.019780,0.003475,0.001253,0.004235,0.003702,...,0.000582,0.000618,0.000202,0.000948,0.045622,0.000948,0.044898,0.000000,0.008525,0.039216
A2ML1,0.025432,0.025432,0.002168,0.005085,0.000368,0.000366,0.000267,0.000000,0.002556,0.009996,...,0.000194,0.000206,0.003636,0.002845,0.025432,0.002845,0.067755,0.000000,0.000000,0.000000
A4GALT,0.021743,0.021743,0.016013,0.020339,0.005420,0.005861,0.030473,0.000000,0.016356,0.004813,...,0.024461,0.025562,0.003030,0.001490,0.021743,0.001490,0.014694,0.024133,0.016198,0.032157


## 1.5 Fraction Betweenness

In [ ]:
importlib.reload(metrics_functions)
process_fraction_betweenness = metrics_functions.fraction_betweenness(reactome_proteins_indexes_apid_huri_df, graph_apid_huri)
process_fraction_betweenness_df = pd.DataFrame.from_dict(process_fraction_betweenness)
process_fraction_betweenness_df = process_fraction_betweenness_df.rename(index=dict(zip(list(process_fraction_betweenness_df.index),list(graph_apid_huri.vs['name']))))
process_fraction_betweenness_df.fillna(value=0, inplace=True)
process_fraction_betweenness_df.to_csv('../../data/processed/metrics/process_fraction_betweenness_apid_huri.csv')
print(process_fraction_betweenness_df.shape)
process_fraction_betweenness_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_fraction_betweenness = metrics_functions.fraction_betweenness(disgenet_prot_index_main_comp, graph_apid_huri)
disease_fraction_betweenness_df = pd.DataFrame.from_dict(disease_fraction_betweenness)
disease_fraction_betweenness_df = disease_fraction_betweenness_df.rename(index=dict(zip(list(disease_fraction_betweenness_df.index),list(graph_apid_huri.vs['name']))))
disease_fraction_betweenness_df.fillna(value=0, inplace=True)
disease_fraction_betweenness_df.to_csv('../../data/processed/metrics/disease_fraction_betweenness_apid_huri.csv')
print(disease_fraction_betweenness_df.shape)
disease_fraction_betweenness_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_fraction_betweenness_conservative = metrics_functions.fraction_betweenness(disgenet_prot_index_conservative_module, graph_apid_huri)
disease_fraction_betweenness_conservative_df = pd.DataFrame.from_dict(disease_fraction_betweenness)
disease_fraction_betweenness_conservative_df = disease_fraction_betweenness_conservative_df.rename(index=dict(zip(list(disease_fraction_betweenness_conservative_df.index),list(graph_apid_huri.vs['name']))))
disease_fraction_betweenness_conservative_df.fillna(value=0, inplace=True)
disease_fraction_betweenness_conservative_df.to_csv('../../data/processed/metrics/disease_fraction_betweenness_conservative_apid_huri.csv')
print(disease_fraction_betweenness_conservative_df.shape)
disease_fraction_betweenness_conservative_df.head()

C:\Users\alexf\OneDrive\Ambiente de Trabalho\Faculdade\02_Mestrado\2Ano\Code\python\src\features\metrics_functions.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processes_df['protein_index'] = processes_df['protein_index']
C:\Users\alexf\OneDrive\Ambiente de Trabalho\Faculdade\02_Mestrado\2Ano\Code\python\src\features\metrics_functions.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processes_df['proteins_ids'] = processes_df['proteins_ids']


  0%|          | 0/5320 [00:00<?, ?it/s]

## 1.6 Random Walks with Restart

In [ ]:
importlib.reload(metrics_functions)
process_rwr = metrics_functions.random_walk_restart(graph_apid_huri, reactome_proteins_indexes_apid_huri_df)
process_rwr_df = pd.DataFrame.from_dict(process_rwr)
process_rwr_df = process_rwr_df.rename(index=dict(zip(list(process_rwr_df.index),list(graph_apid_huri.vs['name']))))
process_rwr_df.to_csv('../../data/processed/metrics/process_rwr_apid_huri.csv')
print(process_rwr_df.shape)
process_rwr_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_rwr = metrics_functions.random_walk_restart(graph_apid_huri, disgenet_prot_index_main_comp)
disease_rwr_df = pd.DataFrame.from_dict(disease_rwr)
disease_rwr_df = disease_rwr_df.rename(index=dict(zip(list(disease_rwr_df.index),list(graph_apid_huri.vs['name']))))
disease_rwr_df.to_csv('../data/metrics/disease_rwr_apid_huri.csv')
print(disease_rwr_df.shape)
disease_rwr_df.head()

In [ ]:
importlib.reload(metrics_functions)
disease_rwr_conservative = metrics_functions.random_walk_restart(graph_apid_huri, disgenet_prot_index_conservative_module)
disease_rwr_conservative_df = pd.DataFrame.from_dict(disease_rwr_conservative)
disease_rwr_conservative_df = disease_rwr_conservative_df.rename(index=dict(zip(list(disease_rwr_conservative_df.index),list(graph_apid_huri.vs['name']))))
disease_rwr_conservative_df.to_csv('../../data/processed/metrics/disease_rwr_conservative_apid_huri.csv')
print(disease_rwr_conservative_df.shape)
disease_rwr_conservative_df.head()

## 1.7 Network Reductions

In [ ]:
importlib.reload(metrics_functions)
ppi80_hyper, ppi80_closeness, ppi80_betweenness, ppi80_rwr, ppi80_fraction_betweeenness = metrics_functions.multiple_metrics(ppi_80, reactome_proteins_indexes_apid_huri_df)
ppi80_hyper.to_csv('../../data/processed/metrics/process_ppi80_hyper_apid_huri.csv')
ppi80_closeness.to_csv('../../data/processed/metrics/process_ppi80_closeness_apid_huri.csv')
ppi80_betweenness.to_csv('../../data/processed/metrics/process_ppi80_betweenness_apid_huri.csv')
ppi80_rwr.to_csv('../../data/processed/metrics/process_ppi80_rwr_apid_huri.csv')
ppi80_fraction_betweenness.to_csv('../../data/processed/process_ppi80_fraction_betweenness_apid_huri.csv')

In [ ]:
importlib.reload(metrics_functions)
protein80_hyper, protein80_closeness, protein80_betweenness, protein80_rwr, protein80_fraction_betweenness = metrics_functions.multiple_metrics(ppi_80_protein, reactome_proteins_indexes_apid_huri_df)
protein80_hyper.to_csv('../../data/processed/metrics/process_protein80_hyper_apid_huri.csv')
protein80_closeness.to_csv('../../data/processed/metrics/process_protein80_closeness_apid_huri.csv')
protein80_betweenness.to_csv('../../data/processed/metrics/process_protein80_betweenness_apid_huri.csv')
protein80_rwr.to_csv('../../data/processed/metrics/process_protein80_rwr_apid_huri.csv')
protein80_fraction_betweenness.to_csv('../../data/processed/metrics/process_protein80_fraction_betweenness_apid_huri.csv')

In [ ]:
importlib.reload(metrics_functions)
disease_ppi80_hyper, disease_ppi80_closeness, disease_ppi80_betweenness, disease_ppi80_rwr, disease_ppi80_fraction_betweeenness = metrics_functions.multiple_metrics(ppi_80, disgenet_prot_index_main_comp)
disease_ppi80_hyper.to_csv('../../data/processed/metrics/disease_ppi80_hyper_apid_huri.csv')
disease_ppi80_closeness.to_csv('../../data/processed/metrics/disease_ppi80_closeness_apid_huri.csv')
disease_ppi80_betweenness.to_csv('../../data/processed/metrics/disease_ppi80_betweenness_apid_huri.csv')
disease_ppi80_rwr.to_csv('../../data/processed/metrics/disease_ppi80_rwr_apid_huri.csv')
disease_ppi80_fraction_betweenness.to_csv('../../data/processed/disease_ppi80_fraction_betweenness_apid_huri.csv')

In [ ]:
importlib.reload(metrics_functions)
disease_protein80_hyper, disease_protein80_closeness, disease_protein80_betweenness, disease_protein80_rwr, disease_protein80_fraction_betweeenness = metrics_functions.multiple_metrics(ppi_80_protein, disgenet_prot_index_main_comp)
disease_protein80_hyper.to_csv('../../data/processed/metrics/disease_protein80_hyper_apid_huri.csv')
disease_protein80_closeness.to_csv('../../data/processed/metrics/disease_protein80_closeness_apid_huri.csv')
disease_protein80_betweenness.to_csv('../../data/processed/metrics/disease_protein80_betweenness_apid_huri.csv')
disease_protein80_rwr.to_csv('../../data/processed/metrics/disease_protein80_rwr_apid_huri.csv')
disease_protein80_fraction_betweenness.to_csv('../../data/processed/disease_protein80_fraction_betweenness_apid_huri.csv')

In [ ]:
importlib.reload(metrics_functions)
disease_ppi80_hyper_conservative, disease_ppi80_closeness_conservative, disease_ppi80_betweenness_conservative, disease_ppi80_rwr_conservative, disease_ppi80_fraction_betweeenness_conservative = metrics_functions.multiple_metrics(ppi_80, disgenet_prot_index_conservative_module)
disease_ppi80_hyper_conservative.to_csv('../../data/processed/metrics/disease_ppi80_hyper_conservative_apid_huri.csv')
disease_ppi80_closeness_conservative.to_csv('../../data/processed/metrics/disease_ppi80_closeness_conservative_apid_huri.csv')
disease_ppi80_betweenness_conservative.to_csv('../../data/processed/metrics/disease_ppi80_betweenness_conservative_apid_huri.csv')
disease_ppi80_rwr_conservative.to_csv('../../data/processed/metrics/disease_ppi80_rwr_conservative_apid_huri.csv')
disease_ppi80_fraction_betweenness_conservative.to_csv('../../data/processed/disease_ppi80_fraction_betweenness_conservative_apid_huri.csv')

In [ ]:
importlib.reload(metrics_functions)
disease_protein80_hyper_conservative, disease_protein80_closeness_conservative, disease_protein80_betweenness_conservative, disease_protein80_rwr_conservative, disease_protein80_fraction_betweeenness_conservative = metrics_functions.multiple_metrics(ppi_80_protein, disgenet_prot_index_conservative_module)
disease_protein80_hyper_conservative.to_csv('../../data/processed/metrics/disease_protein80_hyper_conservative_apid_huri.csv')
disease_protein80_closeness_conservative.to_csv('../../data/processed/metrics/disease_protein80_closeness_conservative_apid_huri.csv')
disease_protein80_betweenness_conservative.to_csv('../../data/processed/metrics/disease_protein80_betweenness_conservative_apid_huri.csv')
disease_protein80_rwr_conservative.to_csv('../../data/processed/metrics/disease_protein80_rwr_conservative_apid_huri.csv')
disease_protein80_fraction_betweenness_conservative.to_csv('../../data/processed/disease_protein80_fraction_betweenness_conservative_apid_huri.csv')